### Comparison of all the ABCs considered in the paper
#### Used for Figure 9

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from matplotlib import animation
from IPython.display import HTML
plt.rcParams['text.usetex'] = True
from piston_Lagrangian import setup
from utils import max_error

### Auxiliary functions

In [ ]:
def generate_input_CLAW(method):
    far_field_damping = False 
    scalar_far_field =False
    relaxation_method = False
    matrix_filter=False
    integral_source = False
    euler_RS = 'euler'
    solver_class = 'sharpclaw_custom'
    far_field_damping_rate = 30.0
    damping_function = "Mayer"
    slowing_function = "Mayer"

    if method == "RM":
        #Sponge layer-based absorbing boundary conditions
        #Using scalar weight function
        relaxation_method = True #Sponge layer far_field_damping

    elif method == "RM-SO":
        #Sponge layer-based absorbing boundary conditions
        #Using scalar far_field_damping function + Slowing down operator
        euler_RS = 'euler_slowing'
        relaxation_method = True #Sponge layer far_field_damping

    elif method == "RM-M":
        #Sponge layer-based absorbing boundary conditions
        #Using matrix far_field_damping function
        euler_RS = 'euler'
        relaxation_method = True
        matrix_filter = True

    elif method == "RM-M-SO":
        #Sponge layer-based absorbing boundary conditions
        #Using matrix far_field_damping function + Slowing down operator
        euler_RS = 'euler_slowing'
        relaxation_method = True
        matrix_filter = True
    
    elif method == "SDO":
        #Slowing down and far_field_damping operators
        euler_RS = 'euler_slowing'
        far_field_damping = True
        solver_class = 'sharpclaw'
        slowing_function = "Mayer"
        damping_function = "Engsig-Karup2"

    elif method == "S-SDO":
        #Slowing down and far_field_damping operators
        euler_RS = 'euler_slowing'
        far_field_damping = True 
        scalar_far_field =True
        solver_class = 'sharpclaw'
        slowing_function = "Mayer"
        damping_function = "Engsig-Karup2"
    
    elif method == "NDO":
        #Slowing down and far_field_damping operators
        euler_RS = 'euler'
        far_field_damping = True 
        integral_source = True
        solver_class = 'sharpclaw'
        far_field_damping_rate = 20.0

    elif method == "SNDO":
        #Slowing down and far_field_damping operators
        euler_RS = 'euler_slowing'
        far_field_damping = True 
        integral_source = True
        solver_class = 'sharpclaw'

    
    return (euler_RS, relaxation_method, far_field_damping, matrix_filter,
             scalar_far_field, integral_source, solver_class, far_field_damping_rate,
             damping_function, slowing_function)

In [ ]:
def set_and_run_CLAW(method, data):
    (tfinal,xmax2,mx2,piston_freq,M,
    start_slowing,stop_slowing,
    start_absorbing,stop_absorbing) = data
    (euler_RS, relaxation_method, far_field_damping,
    matrix_filter, scalar_far_field,
    integral_source, solver_class,far_field_damping_rate,
    damping_function, slowing_function) = generate_input_CLAW(method)
    piston_problem2 = setup(tfinal=tfinal, xmax=xmax2, mx=mx2, M=M, CFL=0.8,limiting=1,nout=100,
                        solver_type=solver_class, order=2, time_integrator='Heun',
                        euler_RS=euler_RS, 
                        relaxation_method=relaxation_method,
                        far_field_damping=far_field_damping,
                        far_field_damping_rate=far_field_damping_rate,
                        matrix_filter=matrix_filter,
                        start_slowing=start_slowing, stop_slowing=stop_slowing,
                        start_absorbing=start_absorbing, stop_absorbing=stop_absorbing,
                        piston_freq=piston_freq,
                        sigma_damping_type=damping_function, 
                        sigma_slowing_type=slowing_function,
                        scalar_far_field=scalar_far_field,
                        integral_source=integral_source) #Not really using implicit integrator at the moment
    piston_problem2.verbosity = 0
    piston_problem2.run()
    piston_problem2.solver.__del__()
    return piston_problem2

### Problem setup

In [ ]:
#sponge_layer_width_list = np.round(2*np.pi*np.array([0.25,0.5,1,4]))
def comparison_setup(sponge_layer_width_list,piston_freq=1., p=20, p_domain=30, p_ABC=20, p_domain_ABC=30, M=0.4, 
                    verbose=True, N=200,
                    start_slowing=20*np.pi, #p_ABC*2*np.pi
                    start_absorbing=20*np.pi):
    if verbose:
        print("#################################################################")
        print("RUNNING SIMULATION WITH PARAMETERS:\npiston_freq = ", piston_freq,
         ", p = ", p, ", p_domain = ", p_domain, ", M = ", M)

    #Reference solution
    xmax1 = p_domain*2*np.pi #p*2*np.pi
    mx1 = p_domain*N #Number of grid points
    tfinal = p*2*np.pi

    #Run reference solution
    piston_problem1 = setup(tfinal=tfinal, xmax=xmax1, mx=mx1, M=M, CFL=0.8,limiting=1,solver_type='sharpclaw_custom',
    order=2, time_integrator='Heun', nout=100, piston_freq=piston_freq)
    piston_problem1.verbosity = 0
    piston_problem1.run()
    piston_problem1.solver.__del__()

    #Run "truncated" solutions
    xmax2 = p_domain_ABC*2*np.pi #p*2*np.pi
    mx2 = p_domain_ABC*N #Number of grid points

    #method_list = ["RM","RM-SO","RM-M","RM-M-SO","SDO","S-SDO", "NDO","SNDO"]
    method_list = ["RM","RM-M","S-SDO","SDO", "NDO"]

    error_list=[]
    for width in sponge_layer_width_list:
        if verbose:
            print("Layer width = ", width)
        stop_slowing = start_absorbing + width*2*np.pi
        stop_absorbing = start_absorbing + width*2*np.pi
        data = (tfinal,xmax2,mx2,piston_freq,M,
            start_slowing,stop_slowing,
            start_absorbing,stop_absorbing) 

        x_comparison = min(start_slowing,start_absorbing)
        error_list.append([])
        for method in method_list:
            if verbose:
                print("Running ", method)
            piston_problem2 = set_and_run_CLAW(method, data)
            indx = np.where(piston_problem2.grid.x.centers<=x_comparison)
            error_list[-1].append(max_error(piston_problem1,piston_problem2,indx=indx))
            if not np.allclose(piston_problem1.grid.x.centers[indx],piston_problem2.grid.x.centers[indx]):
                raise ValueError("Grids are not the same!!!")
            if verbose:
                print(method," DONE")
                
    if verbose:
        print("#################################################################")
    return np.array(error_list)

## Getting results in far-field

In [ ]:
#list_sponge_layer_widths = np.round(np.logspace(np.log10(0.01),np.log10(5.),10),3)
list_sponge_layer_widths = np.round(np.logspace(np.log10(0.01),np.log10(10.),11),2)
start_slowing = 20*np.pi
start_absorbing = 20*np.pi

### Running and writing data far-field

In [ ]:
N_per_period = [10, 50, 250]
list_error_arrays_far_field = []
for N in N_per_period:
    error_list = comparison_setup(sponge_layer_width_list=list_sponge_layer_widths , M=0.4,
                                piston_freq=1., p=20, p_domain=30, p_ABC=20, p_domain_ABC=30, N=N,
                                start_slowing=start_slowing, start_absorbing=start_absorbing)
    print("#################################################################")
    print("Error list: ",error_list)
    print("#################################################################")

    list_error_arrays_far_field.append(error_list)
    np.savetxt("./data_all_far_field/N="+str(N)+".txt",error_list)

### Plotting function

In [ ]:
p=20; p_domain=30; p_ABC=20; p_domain_ABC=30
sponge_layer_width_list = list_sponge_layer_widths.copy()


def plot_errors_diff_Nx(list_error_arrays, list_titles, ylim=None, list_ylim=None, logscales=[]):
    #Create a figure with len(error_list_np) axes in a column
    plt.close()
    fig,ax = plt.subplots(len(list_error_arrays),1,figsize=(5,2*len(list_error_arrays)),dpi=300)

    #######################
    #Sorry for hardcoding this
    #Self-convergence error (Self_convergence.ipynb)
    convError = np.array([1.061677543091432, 0.16727682415561385, 0.030993006342411783])
    #Constant extrapolation error (Constant_Extrapolation_Error.ipynb)
    constExtrapError = np.array([0.02572953158743871, 0.022003876484894457, 0.03087486484936671])
    #######################
    
    #method_list = ["RM","RM-SO","RM-M","RM-M-SO","SDO","S-SDO", "NDO","SNDO"]
    method_list = ["RM","RM-M","S-SDO","SDO", "NDO"]
    color_list = ['blue','red','green','purple','brown']#,'orange', 'black', 'cyan']
    marker_list = ['o','o','x','x','+']
    linetype_list = ['solid','dashed','solid','dashed','dotted']#,"dashdot", "dotted","dashed"]
    for i,error_list_np in enumerate(list_error_arrays):
        ax[i].set_xscale('log')
        if i<len(logscales):
            if logscales[i]:
                ax[i].set_yscale('log')
        #remove all minor ticks in x and y axis
        ax[i].xaxis.set_minor_locator(mpl.ticker.NullLocator())
        ax[i].yaxis.set_minor_locator(mpl.ticker.NullLocator())
        # Set the tick locations to be the values from the sponge_layer_width_list
        if i==len(list_error_arrays)-1:
            ax[i].set_xticks(sponge_layer_width_list)
            xtickslabels = np.round(sponge_layer_width_list,2)
            ax[i].set_xticklabels(xtickslabels)
            ax[i].xaxis.set_tick_params(which='minor', bottom=False)
        #Forcing the x-axis to be between 0.23 and 11
        ax[i].set_xlim(sponge_layer_width_list[0],sponge_layer_width_list[-1])
        if ylim is not None:
            ax[i].set_ylim(ylim)
        if list_ylim is not None:
            ax[i].set_ylim(list_ylim[i])
        # Remove the unwanted ticks
        
        ax[i].set_xlabel("Sponge layer width/Wavelength"+r"$(\omega/L$)")

        ax[i].axvline(x=1,linestyle='--',color='black',linewidth=0.5)
        linelist = [] #For the legend
        for j,method in enumerate(method_list):
            line, = ax[i].plot(sponge_layer_width_list,error_list_np[:,j],color=color_list[j],
                               linewidth=2, linestyle=linetype_list[j],label=method,
                               marker=marker_list[j])
            linelist.append(line)
            #Connect the points with a thin line    
        if i==1:
            ax[i].set_ylabel("Error ABC")
        if i==0:
            #ax[i].legend( bbox_to_anchor=(1., 1.05))
            ax[i].legend(handles=linelist, loc='center left')
        ax[i].set_title(list_titles[i])
        hline1, =ax[i].plot(sponge_layer_width_list,
                        convError[i]*np.ones_like(sponge_layer_width_list),
                        linestyle='-.',
                        color='black',marker='^',label="Discretization Error")
        #hline2 = ax[i].hlines(constExtrapError[i],0.01,5,linestyles=':',colors='grey',
                     #label="Constant Extrapolation")
        hline2, =ax[i].plot(sponge_layer_width_list,
                      constExtrapError[i]*np.ones_like(sponge_layer_width_list),
                      linestyle=':',
                      color='grey',marker='*',label="Const. Extrap. Error") 
        handles = [hline1,hline2]
        # if i==0:
        #     ax[i].legend(handles=handles,loc='upper right')
    plt.show()

### Reading and plotting far-field

In [ ]:
N_per_period = [10, 50, 250]
list_error_arrays_far_field = [[] for i in range(len(N_per_period))]


for i,N in enumerate(N_per_period):
    list_error_arrays_far_field[i] = np.loadtxt("./data_all_far_field/N="+str(N)+".txt")

list_titles = ["N = "+str(N) for N in N_per_period]

In [ ]:

plot_errors_diff_Nx(list_error_arrays_far_field, list_titles, ylim=None, list_ylim=None, logscales=[True,True,True])